# DISRA Science Case Module

**NAME:** QSO_SURVEY

**GOAL:** What happens to the Circumgalactic Medium as massive galaxies quench and transform? 

**OBJECTIVE:** Obtain QSO absorption-line measurements for > 100 massive galaxies in from z = 0.5-1.5. 

**CODE PURPOSE:** Process the SDSS/GALEX QSO catalog to optimize the pathlength for z = 0.5 - 1.5 QSO absorbers. For now we are going to do this without any prior knowledge of foreground galaxies. 

**BACKGROUND:**

Over the redshift interval z = 2 to 0.5, the fraction of massive galaxies (>10^10 Msun) that are star forming declines from $\gtrsim 100\%$ to $\lesssim 10\%$, a phenomenon known as `quenching'. This could be called the "epoch of galaxy transformation". Based on data collected at low redshift, we believe that the circumgalactic medium (CGM) of galaxies plays a large role in governing galaxy accretion, and is also the place where galactic feedback in the form of energetic outflows deposits energy and metals. But we do know very little about how the CGM participates in galaxy quenching -- whether it drives quenching by cutting off accretion, or whether it is a passive recipient of internal feedback. This program will use HWO's revolutionary UV capability to extend CGM observations from Hubble, which are limited to $z \lesssim 1$ by its sensitivity, into the $z > 1$ era when masssive galaxies are in their transformational phase. The figure of merit is pathlength per hour, which determines the number of galaxies whose CGM will be probed as a function of time. 
	  
**MODIFICATION HISTORY:**

[2024-05-08] Started by JT in July 2024 


First we will do our basic imports. 

In [ ]:
import matplotlib.pyplot as plt 
from astropy.table import Table
import numpy as np 
import astropy.units as u
import matplotlib.pyplot as plt 
from syotools.wrappers.uvspec import uvspec_snr, uvspec_exptime

Now we read in a QSO catalog, from a qsocat/GALEX cross-match, filter out QSOs that are too faint or too low in redshift, and plot the distribution of QSO FUV mags vs. redshift to get a sense of the distribution of background sources. The QSO catalog is bundled with hwotools in the "data" subdirectory. 

In [ ]:
qsocat = Table.read('../data/dr7qso_galex.fits')
qsocat = qsocat['SDSSNAME', 'RA', 'DEC', 'Z', 'PSFMAG_G', 'PSFMAGERR_G', 'FUV_MAG', 'NUV_MAG']
qsocat['gal_pathlength'] = qsocat['Z'] - 0.5 - 0.1
qsocat['gal_pathlength'][qsocat['gal_pathlength'] < 0.] = 0. 

We will now filter the QSO catalog for z < 2 and FUV_MAG < 20, and plot it. 

In [ ]:
qsocat = qsocat[qsocat['FUV_MAG'] > 0.]
qsocat = qsocat[qsocat['FUV_MAG'] < 20.]
qsocat = qsocat[qsocat['Z'] < 2.]
qsocat = qsocat[qsocat['Z'] > 0.51]
plt.scatter(qsocat['Z'], qsocat['FUV_MAG'], marker='o', s=0.1, color='red') 
qsocat.sort('FUV_MAG') 
plt.ylim(17,21) 
plt.xlim(0, 2.)
plt.xlabel('Redshift z') 
_ = plt.ylabel('QSO FUV Mag') 

Now we will write a function that calls the SYOTools uvspec_exptime wrapper around the UVI ETC. To save time, because there are so many QSOs, we will not compute the unique SNR/exptime for each one. Rather, we will compute the values for a small set of magnitudes and then interpolate the exptime for each individual QSO from this curve. 

In [ ]:
def qso_exptime(qsocat, snr_goal, eac='EAC3'): 
    mag_list = (np.arange(11)*0.5 + 15.)  
    exptime_list = [] 
    for mag in mag_list:  
        wave120, exp120, uvi = uvspec_exptime(eac, 'G120M', 'flat', mag, snr_goal, silent=True) 
        exptime_list.append(exp120[8425]/3600.)   #<---- picked 1150 A somewhat arbitrarily, 3600 converts sec to hours 
        
    qsocat[eac + '_exptime_snr_'+str(snr_goal)] = np.interp(qsocat['FUV_MAG'], mag_list, exptime_list) #---- here is the interpolation. 
    return qsocat 

Now call this function six time to get exposure times for SNR = 10 and 20 for each QSO added to the catalog table. We will do this for all three EACs. 

In [ ]:
qsocat = qso_exptime(qsocat, 10, 'EAC1')   
qsocat = qso_exptime(qsocat, 20, 'EAC1') 

qsocat = qso_exptime(qsocat, 10, 'EAC2')   
qsocat = qso_exptime(qsocat, 20, 'EAC2')  

qsocat = qso_exptime(qsocat, 10, 'EAC3')   
qsocat = qso_exptime(qsocat, 20, 'EAC3')  

Print out the first few lines of the table to ensure things make sense. 

In [ ]:
qsocat[0:10]

Now we will derive this DISRA case's Figure of Merit for targets, which is the cosmic pathlength $\Delta z$ per hour of exposure time. 
We will do this for SNR = 10 and SNR = 20 

In [ ]:
def get_pathlengths_per_hour(qsocat, eac): 
    qsocat[eac+'_pathlength_per_hour_sn10'] = qsocat['gal_pathlength'] / qsocat[eac+'_exptime_snr_10']
    qsocat[eac+'_pathlength_per_hour_sn20'] = qsocat['gal_pathlength'] / qsocat[eac+'_exptime_snr_20']
    qsocat[eac+'_total_exposure_time_sn10'] = qsocat['Z'] * 0.0 
    qsocat[eac+'_total_exposure_time_sn20'] = qsocat['Z'] * 0.0 
    qsocat[eac+'_total_pathlength_sn10'] = qsocat['Z'] * 0.0 
    qsocat[eac+'_total_pathlength_sn20'] = qsocat['Z'] * 0.0 

In [ ]:
get_pathlengths_per_hour(qsocat, 'EAC1') 
get_pathlengths_per_hour(qsocat, 'EAC2') 
get_pathlengths_per_hour(qsocat, 'EAC3') 

Sort the table so that the most valuable targets, those with the highest pathlength per hour of exposure, are ranked highest. Then we'll plot the results. 

In [ ]:
qsocat.sort('EAC1_pathlength_per_hour_sn10', reverse=True)

In [ ]:
plt.scatter(qsocat['Z'], qsocat['EAC1_pathlength_per_hour_sn10'], linestyle='solid', label='SNR = 10')
plt.scatter(qsocat['Z'], qsocat['EAC1_pathlength_per_hour_sn20'], color='orange', label='SNR = 20')
plt.legend() 
plt.xlabel('QSO Redshift')
plt.ylabel('Pathlength per Hour') 
_ = plt.title('Pathlength per Hour, EAC1 only') 

Now we need to get the *cumulative* pathlength for each case, SNR = 10, 20 and the three EACs 

In [ ]:
def get_cumulative_pathlength(qsocat, eac): 

    total_exp_10 = 0.0 
    total_exp_20 = 0.0 
    total_path = 0.

    for row in qsocat:    
        total_exp_10 = total_exp_10 + row[eac+'_exptime_snr_10']   
        row[eac+'_total_exposure_time_sn10'] = total_exp_10
        total_exp_20 = total_exp_20 + row[eac+'_exptime_snr_20']   
        row[eac+'_total_exposure_time_sn20'] = total_exp_20
        total_path = total_path + row['gal_pathlength']
        row[eac+'_total_pathlength_sn10'] = total_path
        row[eac+'_total_pathlength_sn20'] = total_path
        

In [ ]:
get_cumulative_pathlength(qsocat, 'EAC1') 
get_cumulative_pathlength(qsocat, 'EAC2') 
get_cumulative_pathlength(qsocat, 'EAC3') 

Now we plot the total pathlength over total exposure time, so we can see how many hours must be invested to reach a certain pathlength. 

In [ ]:
plt.plot(qsocat['EAC1_total_exposure_time_sn10'], qsocat['EAC1_total_pathlength_sn10'], label='EAC1, SNR = 10', color='blue')
plt.plot(qsocat['EAC1_total_exposure_time_sn20'], qsocat['EAC1_total_pathlength_sn20'], label='EAC1, SNR = 20', color='orange')
plt.plot(qsocat['EAC3_total_exposure_time_sn10'], qsocat['EAC3_total_pathlength_sn10'], linestyle = 'dashed', label='EAC3, SNR = 10', color='blue')
plt.plot(qsocat['EAC3_total_exposure_time_sn20'], qsocat['EAC3_total_pathlength_sn20'], linestyle = 'dashed', label='EAC3, SNR = 20', color='orange')

plt.xlabel('Total Hours')
plt.legend() 
plt.xlim(-10,500) 
plt.ylim(-100,1000) 
plt.title('Total Pathlength vs. Exposure Time') 
_ = plt.ylabel('Total Pathlength') 